In [1]:
# SOLOv2 train  
from importlib import reload


from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import os


def get_timestamp():
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)


#cfg = Config.fromfile('./configs/solov2/solov2_light_512_dcn_r50_fpn_custom.py')
#cfg.load_from = './checkpoints/SOLOv2_LIGHT_512_DCN_R50_3x.pth'
cfg = Config.fromfile('./configs/solov2/solov2_light_448_r50_fpn_8gpu_3x.py') #works
cfg.load_from = './checkpoints/SOLOv2_Light_448_R50_3x.pth'
#cfg.load_from = './checkpoints/epoch_1.pth'

cfg.dataset_type = 'CocoDataset'
PREFIX = os.path.abspath('C:/datasets/blurred')
cfg.data.train.img_prefix = PREFIX + "/color/"
cfg.data.train.ann_file = PREFIX + '/instances_hands_2100.json'
cfg.data.train.type = 'CocoDataset'

# Validation does not work for SOLO (I guess)
# cfg.data.val.img_prefix = PREFIX + "/color/"
# #cfg.data.val.classes = cfg.classes
# cfg.data.val.ann_file = PREFIX + '/instances_hands_train2022.json'
# cfg.data.val.type = 'CocoDataset'

# cfg.data.test.img_prefix = PREFIX + "/color/"
# #cfg.data.test.classes = cfg.classes
# cfg.data.test.ann_file = PREFIX + '/instances_hands_train2022.json'
# cfg.data.test.type = 'CocoDataset'

cfg.optimizer.lr = 0.00001
cfg.lr_config.warmup = None

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device_ids = range(1)
cfg.gpus = 1

cfg.work_dir = "./checkpoints"
cfg.total_epochs = 2
cfg.checkpoint_config = dict(create_symlink=False, interval = 1)
cfg.data.imgs_per_gpu = 16
cfg.data.workers_per_gpu = 1

cfg.log_config = dict(
    interval=1,
    hooks=[
        dict(type='TextLoggerHook'),
        #dict(type='TensorboardLoggerHook')
    ])

model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]
datasets[0].CLASSES = ["hand", "dummy"]
model.CLASSES = datasets[0].CLASSES

# print(f'Config:\n{cfg.pretty_text}')

# CHECK OUT frozen_stages=0 for fine-tunning !!
# try resnext backbone

get_timestamp()
train_detector(model, datasets[0], cfg, distributed=False, validate=False)
get_timestamp()


2022-03-10 11:19:28,694 - mmdet - INFO - load model from: torchvision://resnet50
2022-03-10 11:19:28,695 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-03-10 11:19:28,910 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Current Time = 11:19:28


C:\Users\C201_ALES\anaconda3\envs\solo2\lib\site-packages\mmcv\runner\epoch_based_runner.py:186: UserWarning: Runner was deprecated, please use EpochBasedRunner instead
  'Runner was deprecated, please use EpochBasedRunner instead')
C:\Users\C201_ALES\anaconda3\envs\solo2\lib\site-packages\mmcv\runner\base_runner.py:64: UserWarning: batch_processor is deprecated, please implement train_step() and val_step() in the model instead.
  warnings.warn('batch_processor is deprecated, please implement '
2022-03-10 11:19:30,128 - mmdet - INFO - load checkpoint from local path: ./checkpoints/SOLOv2_Light_448_R50_3x.pth
2022-03-10 11:19:30,249 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.solo_cate.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 256, 3, 3]).
size mismatch for bbox_head.solo_cate.bias: copying a param with shape torch.Size([80]) from checkpoint, the s

Current Time = 11:21:19
